In [5]:
import os
from classeg.utils.constants import PREPROCESSED_ROOT
import glob
import numpy as np

In [4]:
label_path = "/home/andrewheschl/Documents/Dataset/ClassPipeline/preprocessed/Dataset_420/fold_0/train/labelsTr/case_00131.npy"

In [8]:
label = np.load(label_path)
print(label.shape)
print(np.unique(label))

(1, 1024, 1024)
[0. 1.]


In [34]:
label2 = label*2
print(np.unique(label2))

[0. 2.]


In [29]:
np.ceil(np.log2(label2.max())+1)

2.0

In [86]:
# convert to bit representation of the discrete label mask, where channel 0 is the least significant bit
def label_to_bitmask(label):
    label = label.astype(np.uint8)
    
    n_bits = np.floor(np.log2(label.max())).astype(int) + 1
    n_bits = max(n_bits, 1)

    bitmask = np.zeros((n_bits, label.shape[1], label.shape[2]), dtype=np.uint8)
    for i in range(n_bits):
        bitmask[i, ...] = (label[0]//2**i)%2
    return bitmask

def bitmask_to_label(bitmask):
    label = np.zeros((1, bitmask.shape[1], bitmask.shape[2]), dtype=np.uint8)
    for i in range(bitmask.shape[0]):
        label += bitmask[i, ...]*2**i
    return label

In [93]:
bitmask1 = label_to_bitmask(label)
bitmask2 = label_to_bitmask(label2)

print(label.shape, np.unique(label), bitmask1.shape, np.unique(bitmask1))
print(label2.shape, np.unique(label2), bitmask2.shape, np.unique(bitmask2))

(1, 1024, 1024) [0. 1.] (1, 1024, 1024) [0 1]
(1, 1024, 1024) [0. 2.] (2, 1024, 1024) [0 1]


In [94]:
label_returned = bitmask_to_label(bitmask1)
print(np.unique(label_returned), label_returned.shape)
label_returned2 = bitmask_to_label(bitmask2)
print(np.unique(label_returned2), label_returned2.shape)

[0 1] (1, 1024, 1024)
[0 2] (1, 1024, 1024)


In [95]:
print(np.all(label == label_returned))
print(np.all(label2 == label_returned2))

True
True


In [105]:
test_label = np.random.randint(0, 2**2, (1, 4, 4), dtype=np.uint8)
print(test_label.shape, np.unique(test_label))

test_bitmask = label_to_bitmask(test_label)
print(test_bitmask.shape, np.unique(test_bitmask))
print(test_bitmask)
print("==========")
print(test_label)
print(np.all(test_label == bitmask_to_label(test_bitmask)))

(1, 4, 4) [0 1 2 3]
(2, 4, 4) [0 1]
[[[0 0 1 1]
  [1 1 1 1]
  [1 0 0 0]
  [1 0 0 1]]

 [[1 0 0 0]
  [0 0 1 1]
  [1 1 0 0]
  [1 1 1 0]]]
[[[2 0 1 1]
  [1 1 3 3]
  [3 2 0 0]
  [3 2 2 1]]]
True
